<a href="https://colab.research.google.com/github/ed-ntim/D2/blob/master/open_social_ai_edward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
#install ctransformers
!pip install ctransformers[cuda]

#import
from ctransformers import AutoModelForCausalLM

# LLM English Models

We experiment with a number of LLM models:
1. Mistral-7B : Mistral has a context length of 8k.
2. NousHermes-13B : Fine-tuned LlaMa with a context length of 8k.
3. Zephyr-7B : Fine-tuned Mistral with a context length of 8k.

The models are loaded in 4-bit quantized form with the C-Transformers library.

In [ ]:
#load the model from huggingface with 50 layers
llm_mistral = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GGUF",
model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
model_type = "mistral", gpu_layers = 50, context_length = 5000)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

In [ ]:
#load the model from huggingface with 50 gpu layers
llm_noushermes = AutoModelForCausalLM.from_pretrained("TheBloke/Nous-Hermes-13B-GGML",
model_file="nous-hermes-13b.ggmlv3.q2_K.bin",model_type="llama", gpu_layers = 50)

In [ ]:
llm_zephyr = AutoModelForCausalLM.from_pretrained("TheBloke/zephyr-7B-alpha-GGUF",
model_file="zephyr-7b-alpha.Q2_K.gguf",
model_type = "mistral", gpu_layers = 50, context_length = 5000)

In [ ]:
#function to print the model output in colab in a readable manner
def colab_print(text, max_width = 120):
  words = text.split()
  line = ""
  for word in words:
    if len(line) + len(word) + 1 > max_width:
      print(line)
      line = ""
    line += word + " "
  # print (line)
  return line

def get_prompt_lama(instruction):
    # B_INST, E_INST = "<s>[INST]", "[/INST]</s>"
    # B_SYS, E_SYS = "<<SYS>>", "<</SYS>>"
    DEFAULT_SYSTEM_PROMPT = "You live in Trieste. Always keep the conversation going by replying something."
    prompt_template =DEFAULT_SYSTEM_PROMPT+ instruction
    return prompt_template

instruction = """
which universities are in trieste?
"""
prompt_text = get_prompt_lama(instruction)

line  = colab_print(llm_noushermes(prompt_text,
                max_new_tokens = 256,
                temperature = 0.9,
                top_k = 55,
                top_p = 0.93,
                repetition_penalty = 1.2,
                ))
print(line)

In [ ]:
instruction = """
which universities are in trieste?
"""

for i in range(5):
  prompt_text = get_prompt_lama(instruction)

  instruction  = colab_print(llm_noushermes(prompt_text,
                  max_new_tokens = 256,
                  temperature = 0.9,
                  top_k = 55,
                  top_p = 0.93,
                  repetition_penalty = 1.2,
                ))
  print(instruction)
  print("Response:")

In [ ]:
instruction = """
which universities are in trieste?
"""

for i in range(5):
  prompt_text = get_prompt_mistral(instruction)

  instruction  = colab_print(llm_mistral(prompt_text,
                  max_new_tokens = 256,
                  temperature = 0.9,
                  top_k = 55,
                  top_p = 0.93,
                  repetition_penalty = 1.2,
                ))
  print("Mistral:")
  print(instruction)

  prompt_text = get_prompt_lama(instruction)

  instruction  = colab_print(llm_noushermes(prompt_text,
                  max_new_tokens = 256,
                  temperature = 0.9,
                  top_k = 55,
                  top_p = 0.93,
                  repetition_penalty = 1.2,
                ))

  print("Llama:")
  print(instruction)